In [64]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [65]:
import unicodedata, re
from tqdm.notebook import tqdm

In [66]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(insta_url)
time.sleep(3)

### 로그인하기

In [67]:
with open('password.txt') as f:
    password = f.read()

In [68]:
id_ = 'vvbshvv'
input_id_=driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_id_.clear()
input_id_.send_keys(id_)

KeyboardInterrupt: 

In [ ]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)

In [ ]:
input_pwd.submit()
time.sleep(2)

In [ ]:
#로그인 정보 저장 - 나중에하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [ ]:
#알림설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 제주맛집으로 검색

In [ ]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [ ]:
driver.get(url)
time.sleep(5)

In [ ]:
# 첫번째 게시글 클릭
driver.find_element_by_css_selector('._9AhH0').click()
time.sleep(2)


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 본문내용가져오기
try:
    content = soup.select_one('.C4VMK > span').text
    content = unicodedata.normalize('NFC',content)
except:
    content = ' '
content

'제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집"성산을본갈치"성산과 우도를 한눈에 볼수있는 뷰맛집 🏞문어 전복 딱새우 갈치 등등싱싱하고 다양한 해산물이 넘치도록 쏟아진다🦀🐙🦐🐟🦞🦑비주얼 끝판왕 성산 찐맛집 😋살이통통한 갈치를 맛보시려면성산을본갈치로 GoGoGo~~~❗상호 : #성산을본갈치주소 : 서귀포시 성산중앙로64번길5-4 2층영업시간 : 연중무휴 09:00~21:00전화 : 064 - 782 - 6868'

In [ ]:
# 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+',content)
tags

['#성산을본갈치주소']

In [ ]:
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-07'

In [ ]:
# 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [ ]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [ ]:
row = [content, date, like, place, tags]

In [ ]:
#첫번째 게시글에서 다음글로이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(2)

In [ ]:
# 두번째 게시글에서 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [ ]:
# pop up화면을 없앰(x를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
driver.close()

# 데이터프레임으로 만들고 중복 제거

In [69]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df

,content,date,like,place,tags
0,"제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집""성산을본갈치""성산...",2021-12-07,0,,[#성산을본갈치주소]
1,"🤍⁣⁣❣ 제주 서귀포 회맛집, ""김선장회센타"" ❣⁣⁣김선장SET으로 즐기는 다양하고...",2021-12-07,0,,"[#제주맛집, #제주회맛집, #제주회포장, #서귀포핫플, #서귀포횟집, #제주도횟집..."
2,광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 7...,2021-12-06,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
3,.🌅제주 사계해변🌅제주에서 간조 때만 볼 수 있는 한국의 그랜드캐니언!😍이국적인 분...,2021-12-08,"1,065",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
4,#제주맛집🍜제주도민선정맛집 ! 제주순메밀막국수🍜🎄직접 짜서 만드는 들기름으로 건강 ...,2021-12-08,0,제주순메밀막국수,"[#제주맛집🍜제주도민선정맛집, #카멜리아힐맛집.밀가루, #안덕면맛집, #제주도서쪽맛..."
...,...,...,...,...,...
495,줄서서 먹어도 행복하다는 역대급 수제버거로인기몰이중인! 성산 #브라보비치 🍔😋주문 ...,2021-12-08,저 좋아요를 눌러보세,브라보비치-제주,"[#브라보비치, #제주도카페, #제주여행, #제주카페, #여행에미치다_제주, #제주..."
496,(광고)인스타 광고 전문업체 플랜비컴퍼니입니다.현재 수백개의 업체 인스타 광고를 통...,2021-12-08,저 좋아요를 눌러보세,,"[#제주맛집, #제주도맛집, #부산맛집, #서면맛집, #천안맛집, #성산일출봉맛집,..."
497,(광고)인스타 광고 전문업체 플랜비컴퍼니입니다.현재 수백개의 업체 인스타 광고를 통...,2021-12-08,2,,"[#제주맛집, #제주도맛집, #부산맛집, #서면맛집, #천안맛집, #성산일출봉맛집,..."
498,(광고)인스타 광고 전문업체 플랜비컴퍼니입니다.현재 수백개의 업체 인스타 광고를 통...,2021-12-08,7,,"[#제주맛집, #제주도맛집, #부산맛집, #서면맛집, #천안맛집, #성산일출봉맛집,..."


In [112]:
def func(s) :
    
    if '주소' in s :
        i= s.find('주소')
        return s[i:].split('\n')[0]
    else:
        return '-'

def func1(s) :
    if s.endwith

df.content.apply(func).head(50)

0     주소 : 서귀포시 성산중앙로64번길5-4 2층영업시간 : 연중무휴 09:00~21:...
1                                                     -
2     주소 : 제주시 애월읍 애월해안로 752🔸영업 : 오전11시~오후10시🔸전화: 06...
3     주소: 제주 서귀포시 안덕면 사계리..#제주맛집 #제주도맛집 #제주공항근처맛집 #제...
4     주소 : 제주 서귀포시 안덕면 서광리 1413-3전화 : 064-792-0600 📞...
5                                                     -
6                                                     -
7                                                     -
8                                                     -
9                                                     -
10                                                    -
11                                                    -
12                                                    -
13                                                    -
14                                                    -
15                                                    -
16                                                    -
17                                              

0       True
1      False
2       True
3       True
4       True
       ...  
486    False
487    False
491    False
494    False
499    False
Name: content, Length: 325, dtype: bool

In [70]:
# 중복된 값 제거
df.drop_duplicates(subset='content', keep='first', inplace= True)
df.shape

(325, 5)

In [71]:
df.to_csv('인스타-제주맛집.csv', index=False)

In [72]:
df[['place','date']]
df.place.count()

325